*** 
# CHAPTER 7. Hierarchical Query
***
### KEY WORD
* start with : 시작행을 지정 (100번사원 반드시 출력)
* connect by 부모행과 자식행의 관계를 지정

## 7-1.  CONNECT BY .. .PRIOR
### < CONNECT BY *PRIOR* employee_id = manager_id >
* top-down 방식 
* start with를 기준으로 부하직원찾기

In [1]:
import cx_Oracle
import pandas as pd
xedb = cx_Oracle.connect('hr/hr@localhost/xe')
cur = xedb.cursor()

In [2]:
df = pd.read_sql("""
        SELECT employee_id, last_name, manager_id
        FROM employees
        START WITH employee_id=100
        CONNECT BY PRIOR employee_id=manager_id
        """,xedb)
print(df)

     EMPLOYEE_ID    LAST_NAME  MANAGER_ID
0            100         King         NaN
1            101      Kochhar       100.0
2            108    Greenberg       101.0
3            109       Faviet       108.0
4            110         Chen       108.0
5            111      Sciarra       108.0
6            112        Urman       108.0
7            113         Popp       108.0
8            200       Whalen       101.0
9            203       Mavris       101.0
10           204         Baer       101.0
11           205      Higgins       101.0
12           206        Gietz       205.0
13           102      De Haan       100.0
14           103       Hunold       102.0
15           104        Ernst       103.0
16           105       Austin       103.0
17           106    Pataballa       103.0
18           107      Lorentz       103.0
19           114     Raphaely       100.0
20           115         Khoo       114.0
21           116        Baida       114.0
22           117       Tobias     

### < CONNECT BY employee_id = *PRIOR* manager_id >
* bottom up 방식
* start with 기준으로 선임찾기

In [3]:
df = pd.read_sql("""
        SELECT employee_id, last_name, manager_id
        FROM employees
        START WITH employee_id=108
        CONNECT BY employee_id= PRIOR manager_id
        """,xedb)
print(df)

   EMPLOYEE_ID  LAST_NAME  MANAGER_ID
0          108  Greenberg       101.0
1          101    Kochhar       100.0
2          100       King         NaN


## 7-2. LEVEL 

In [4]:
df = pd.read_sql("""
        SELECT LEVEL, last_name
        FROM employees
        START WITH employee_id = 100
        CONNECT BY PRIOR employee_id = manager_id
        """,xedb)
print(df)

     LEVEL    LAST_NAME
0        1         King
1        2      Kochhar
2        3    Greenberg
3        4       Faviet
4        4         Chen
5        4      Sciarra
6        4        Urman
7        4         Popp
8        3       Whalen
9        3       Mavris
10       3         Baer
11       3      Higgins
12       4        Gietz
13       2      De Haan
14       3       Hunold
15       4        Ernst
16       4       Austin
17       4    Pataballa
18       4      Lorentz
19       2     Raphaely
20       3         Khoo
21       3        Baida
22       3       Tobias
23       3       Himuro
24       3   Colmenares
25       2        Weiss
26       3        Nayer
27       3  Mikkilineni
28       3       Landry
29       3       Markle
..     ...          ...
77       2     Partners
78       3         King
79       3        Sully
80       3       McEwen
81       3        Smith
82       3        Doran
83       3       Sewall
84       2    Errazuriz
85       3      Vishney
86       3      

### < 들여쓰기를 이용한 가공 >

In [5]:
df = pd.read_sql("""
        SELECT last_name||lpad(' ', 2 * LEVEL - 2, ' ')AS ename, LEVEL
        FROM employees
        START WITH employee_id = 100
        CONNECT BY PRIOR employee_id = manager_id
        """,xedb)
print(df)

               ENAME  LEVEL
0               King      1
1          Kochhar        2
2      Greenberg          3
3       Faviet            4
4         Chen            4
5      Sciarra            4
6        Urman            4
7         Popp            4
8         Whalen          3
9         Mavris          3
10          Baer          3
11       Higgins          3
12       Gietz            4
13         De Haan        2
14        Hunold          3
15       Ernst            4
16      Austin            4
17   Pataballa            4
18     Lorentz            4
19        Raphaely        2
20          Khoo          3
21         Baida          3
22        Tobias          3
23        Himuro          3
24    Colmenares          3
25           Weiss        2
26         Nayer          3
27   Mikkilineni          3
28        Landry          3
29        Markle          3
..               ...    ...
77        Partners        2
78          King          3
79         Sully          3
80        McEwen    

## 7-3.  ORDER SIBLINGS BY  
같은 부모 아래 형제끼리 정렬 

In [6]:
# 일반 ORDER BY로 정렬을 하게 되면 계층구조가 어긋난다.

df = pd.read_sql("""
        SELECT last_name||lpad(' ', 2 * LEVEL - 2, ' ')AS ename, LEVEL
        FROM employees
        START WITH employee_id = 100
        CONNECT BY PRIOR employee_id = manager_id
        ORDER BY last_name
        """,xedb)
print(df)

              ENAME  LEVEL
0          Abel          3
1          Ande          3
2      Atkinson          3
3      Austin            4
4          Baer          3
5         Baida          3
6         Banda          3
7         Bates          3
8          Bell          3
9     Bernstein          3
10       Bissot          3
11        Bloom          3
12         Bull          3
13       Cabrio          3
14      Cambrault        2
15    Cambrault          3
16       Chen            4
17        Chung          3
18   Colmenares          3
19       Davies          3
20        De Haan        2
21    Dellinger          3
22        Dilly          3
23        Doran          3
24      Ernst            4
25      Errazuriz        2
26      Everett          3
27     Faviet            4
28          Fay          3
29       Feeney          3
..              ...    ...
77        Patel          3
78      Perkins          3
79   Philtanker          3
80       Popp            4
81         Rajs          3
8

In [7]:
# ORDER SIBLINGS BY를 이용하여 계층을 유지한 상태로 정렬

df = pd.read_sql("""
        SELECT last_name||lpad(' ', 2 * LEVEL - 2, ' ')AS ename, LEVEL
        FROM employees
        START WITH employee_id = 100
        CONNECT BY PRIOR employee_id = manager_id
        ORDER SIBLINGS BY last_name
        """,xedb)
print(df)

               ENAME  LEVEL
0               King      1
1        Cambrault        2
2          Bates          3
3          Bloom          3
4            Fox          3
5          Kumar          3
6           Ozer          3
7          Smith          3
8          De Haan        2
9         Hunold          3
10      Austin            4
11       Ernst            4
12     Lorentz            4
13   Pataballa            4
14       Errazuriz        2
15          Ande          3
16         Banda          3
17        Greene          3
18           Lee          3
19       Marvins          3
20       Vishney          3
21           Fripp        2
22      Atkinson          3
23        Bissot          3
24          Bull          3
25        Cabrio          3
26     Dellinger          3
27        Marlow          3
28         Olson          3
29      Sarchand          3
..               ...    ...
77     Cambrault          3
78          Hall          3
79         Olsen          3
80        Tucker    

## 7-4. SYS_CONNECT_BY_PATH 
계층구조 전체경로

In [8]:
df = pd.read_sql("""
        SELECT rpad(sys_connect_by_path(last_name,'/'),32,' ') AS path_1,
                rpad(ltrim(sys_connect_by_path(last_name,'/'),'/'),32,' ') AS path_2
        FROM employees
        START WITH employee_id = 100
        CONNECT BY PRIOR employee_id = manager_id
        """,xedb)
print(df)

                               PATH_1                            PATH_2
0    /King                             King                            
1    /King/Kochhar                     King/Kochhar                    
2    /King/Kochhar/Greenberg           King/Kochhar/Greenberg          
3    /King/Kochhar/Greenberg/Faviet    King/Kochhar/Greenberg/Faviet   
4    /King/Kochhar/Greenberg/Chen      King/Kochhar/Greenberg/Chen     
5    /King/Kochhar/Greenberg/Sciarra   King/Kochhar/Greenberg/Sciarra  
6    /King/Kochhar/Greenberg/Urman     King/Kochhar/Greenberg/Urman    
7    /King/Kochhar/Greenberg/Popp      King/Kochhar/Greenberg/Popp     
8    /King/Kochhar/Whalen              King/Kochhar/Whalen             
9    /King/Kochhar/Mavris              King/Kochhar/Mavris             
10   /King/Kochhar/Baer                King/Kochhar/Baer               
11   /King/Kochhar/Higgins             King/Kochhar/Higgins            
12   /King/Kochhar/Higgins/Gietz       King/Kochhar/Higgins/Giet

# [ 연습문제 ]
### (1) 하향식 탐색한 후에 Higgins 사원 제거 후, 하위 행 처리

In [9]:
# 제거 전 11번째 행 Higgins

df = pd.read_sql("""
        SELECT LEVEL, lpad(' ', 2 * LEVEL - 2, ' ') || last_name AS ename
        FROM employees
        START WITH employee_id = 100
        CONNECT BY PRIOR employee_id = manager_id
        """,xedb)
print(df)

     LEVEL            ENAME
0        1             King
1        2          Kochhar
2        3        Greenberg
3        4           Faviet
4        4             Chen
5        4          Sciarra
6        4            Urman
7        4             Popp
8        3           Whalen
9        3           Mavris
10       3             Baer
11       3          Higgins
12       4            Gietz
13       2          De Haan
14       3           Hunold
15       4            Ernst
16       4           Austin
17       4        Pataballa
18       4          Lorentz
19       2         Raphaely
20       3             Khoo
21       3            Baida
22       3           Tobias
23       3           Himuro
24       3       Colmenares
25       2            Weiss
26       3            Nayer
27       3      Mikkilineni
28       3           Landry
29       3           Markle
..     ...              ...
77       2         Partners
78       3             King
79       3            Sully
80       3          

In [14]:
df = pd.read_sql_query(
        "SELECT LEVEL, lpad(' ', 2 * LEVEL - 2, ' ') || last_name AS ename\
        FROM employees\
        WHERE last_name != 'Higgins'\
        START WITH employee_id = 100\
        CONNECT BY PRIOR employee_id = manager_id",xedb)
print(df)

     LEVEL            ENAME
0        1             King
1        2          Kochhar
2        3        Greenberg
3        4           Faviet
4        4             Chen
5        4          Sciarra
6        4            Urman
7        4             Popp
8        3           Whalen
9        3           Mavris
10       3             Baer
11       4            Gietz
12       2          De Haan
13       3           Hunold
14       4            Ernst
15       4           Austin
16       4        Pataballa
17       4          Lorentz
18       2         Raphaely
19       3             Khoo
20       3            Baida
21       3           Tobias
22       3           Himuro
23       3       Colmenares
24       2            Weiss
25       3            Nayer
26       3      Mikkilineni
27       3           Landry
28       3           Markle
29       3           Taylor
..     ...              ...
76       2         Partners
77       3             King
78       3            Sully
79       3          

Higgins 사원의 하위행인 Gietz 사원이 존재
### (2) Higgins 사원 및 하위 행 모두 제거

In [10]:
df = pd.read_sql("""
        SELECT LEVEL, lpad(' ', 2 * LEVEL - 2, ' ') || last_name AS ename
        FROM employees
        START WITH employee_id = 100
        CONNECT BY PRIOR employee_id = manager_id
        AND last_name != 'Higgins'
        """,xedb)
print(df)

     LEVEL            ENAME
0        1             King
1        2          Kochhar
2        3        Greenberg
3        4           Faviet
4        4             Chen
5        4          Sciarra
6        4            Urman
7        4             Popp
8        3           Whalen
9        3           Mavris
10       3             Baer
11       2          De Haan
12       3           Hunold
13       4            Ernst
14       4           Austin
15       4        Pataballa
16       4          Lorentz
17       2         Raphaely
18       3             Khoo
19       3            Baida
20       3           Tobias
21       3           Himuro
22       3       Colmenares
23       2            Weiss
24       3            Nayer
25       3      Mikkilineni
26       3           Landry
27       3           Markle
28       3           Taylor
29       3           Fleaur
..     ...              ...
75       2         Partners
76       3             King
77       3            Sully
78       3          

In [11]:
cur.close()
xedb.close()